In [2]:
import acquire_telco
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import math

In [3]:
raw_data = acquire_telco.new_telco_data()
df = raw_data.copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   contract_type_id          7043 non-null   int64  
 1   payment_type_id           7043 non-null   int64  
 2   internet_service_type_id  7043 non-null   int64  
 3   customer_id               7043 non-null   object 
 4   gender                    7043 non-null   object 
 5   senior_citizen            7043 non-null   int64  
 6   partner                   7043 non-null   object 
 7   dependents                7043 non-null   object 
 8   tenure                    7043 non-null   int64  
 9   phone_service             7043 non-null   object 
 10  multiple_lines            7043 non-null   object 
 11  online_security           7043 non-null   object 
 12  online_backup             7043 non-null   object 
 13  device_protection         7043 non-null   object 
 14  tech_sup

## start cleaning up

In [4]:
# converte the total charge from object to float.

df["total_charges"] = pd.to_numeric(df["total_charges"], errors="coerce")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   contract_type_id          7043 non-null   int64  
 1   payment_type_id           7043 non-null   int64  
 2   internet_service_type_id  7043 non-null   int64  
 3   customer_id               7043 non-null   object 
 4   gender                    7043 non-null   object 
 5   senior_citizen            7043 non-null   int64  
 6   partner                   7043 non-null   object 
 7   dependents                7043 non-null   object 
 8   tenure                    7043 non-null   int64  
 9   phone_service             7043 non-null   object 
 10  multiple_lines            7043 non-null   object 
 11  online_security           7043 non-null   object 
 12  online_backup             7043 non-null   object 
 13  device_protection         7043 non-null   object 
 14  tech_sup

In [47]:
# Drop any unnecessary, unhelpful, or duplicated columns. This could mean dropping foreign key columns but keeping the corresponding string values, for example.
print(df.shape)
df = df.drop_duplicates()
print(df.shape)

(7043, 24)
(7043, 24)


In [48]:
# Find columns with missing values and the total of missing values.
missing = df.isnull().sum()
missing[missing > 0]

total_charges    11
dtype: int64

In [49]:
# drop the null value
print(df.shape)
df = df.dropna()
print(df.shape)

(7043, 24)
(7032, 24)


In [50]:
# columns to drop
df.drop(columns=['payment_type_id', 'internet_service_type_id', 'contract_type_id'], inplace=True)

In [51]:
df.shape

(7032, 21)

In [52]:
# Encode the categorical columns. Create dummy variables of the categorical columns and concatenate them onto the dataframe.

# encode binary categorical variables into numeric values
df['gender_encoded_(female=1)'] = df.gender.map({'Female': 1, 'Male': 0})
df['partner_encoded'] = df.partner.map({'Yes': 1, 'No': 0})
df['dependents_encoded'] = df.dependents.map({'Yes': 1, 'No': 0})
df['phone_service_encoded'] = df.phone_service.map({'Yes': 1, 'No': 0})
df['paperless_billing_encoded'] = df.paperless_billing.map({'Yes': 1, 'No': 0})
df['churn_encoded'] = df.churn.map({'Yes': 1, 'No': 0})
df['online_security_bool'] = df.online_security.map({'Yes': 1, 'No': 0, 'No internet service': 0})
df['online_backup_bool'] = df.online_backup.map({'Yes': 1, 'No': 0, 'No internet service': 0})

In [53]:
# Get dummies for non-binary categorical variables
df_dummy = pd.get_dummies(df[['multiple_lines', \
                              'online_security', \
                              'online_backup', \
                              'device_protection', \
                              'tech_support', \
                              'streaming_tv', \
                              'streaming_movies', \
                              'contract_type', \
                              'internet_service_type', \
                              'payment_type']], dummy_na=False, drop_first=True)
df_dummy.head()

,multiple_lines_No phone service,multiple_lines_Yes,online_security_No internet service,online_security_Yes,online_backup_No internet service,online_backup_Yes,device_protection_No internet service,device_protection_Yes,tech_support_No internet service,tech_support_Yes,...,streaming_tv_Yes,streaming_movies_No internet service,streaming_movies_Yes,contract_type_One year,contract_type_Two year,internet_service_type_Fiber optic,internet_service_type_None,payment_type_Credit card (automatic),payment_type_Electronic check,payment_type_Mailed check
0,0,0,0,0,0,1,0,0,0,1,...,1,0,0,1,0,0,0,0,0,1
1,0,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,1,0
3,0,0,0,0,0,1,0,1,0,0,...,1,0,1,0,0,1,0,0,1,0
4,0,0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,1,0,0,0,1


In [54]:
df.head()

,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,...,payment_type,contract_type,gender_encoded_(female=1),partner_encoded,dependents_encoded,phone_service_encoded,paperless_billing_encoded,churn_encoded,online_security_bool,online_backup_bool
0,0002-ORFBO,Female,0,Yes,Yes,9,Yes,No,No,Yes,...,Mailed check,One year,1,1,1,1,1,0,0,1
1,0003-MKNFE,Male,0,No,No,9,Yes,Yes,No,No,...,Mailed check,Month-to-month,0,0,0,1,0,0,0,0
2,0004-TLHLJ,Male,0,No,No,4,Yes,No,No,No,...,Electronic check,Month-to-month,0,0,0,1,1,1,0,0
3,0011-IGKFF,Male,1,Yes,No,13,Yes,No,No,Yes,...,Electronic check,Month-to-month,0,1,0,1,1,1,0,1
4,0013-EXCHZ,Female,1,Yes,No,3,Yes,No,No,No,...,Mailed check,Month-to-month,1,1,0,1,1,1,0,0


In [55]:
df = pd.concat([df, df_dummy],axis=1)
df.shape

(7032, 50)

In [63]:
# encode number_relationships by utilizing information from dependents_encoded and partner_encoded
df['number_relationships'] = df['dependents_encoded'] + df['partner_encoded']

# encode number_online_services by utilizing information from online_security_encoded and online_backup_encoded
df['number_online_services'] = df['online_security_bool'] + df['online_backup_bool']

# encode tenure in years (rounded down) by utilizing information from tenure (currently stored in months)
df['yearly_tenure'] = df.tenure.apply(lambda x: math.floor(x/12))

# encode has_internet
df['has_internet'] = df.internet_service_type.apply(lambda x: 0 if x == 'None' else 1)

# make another column for additional online services
df['additional_services'] = (df[['online_security','online_backup', 'device_protection', 'tech_support', 'streaming_tv',
                                 'streaming_movies']] == 'Yes').sum(axis=1)

In [64]:
df.shape

(7032, 54)

In [1]:
def clean_telco_data(df):
    df["total_charges"] = pd.to_numeric(df["total_charges"], errors="coerce") #conver the total charge into float
    df = df.drop_duplicates()
    df = df.dropna() # drop 11 row that total charge are null
    df.drop(columns=['payment_type_id', 'internet_service_type_id', 'contract_type_id'], inplace=True)
    
    # encode binary categorical variables into numeric values
    df['gender_encoded_(female=1)'] = df.gender.map({'Female': 1, 'Male': 0})
    df['partner_encoded'] = df.partner.map({'Yes': 1, 'No': 0})
    df['dependents_encoded'] = df.dependents.map({'Yes': 1, 'No': 0})
    df['phone_service_encoded'] = df.phone_service.map({'Yes': 1, 'No': 0})
    df['paperless_billing_encoded'] = df.paperless_billing.map({'Yes': 1, 'No': 0})
    df['churn_encoded'] = df.churn.map({'Yes': 1, 'No': 0})
    df['online_security_bool'] = df.online_security.map({'Yes': 1, 'No': 0, 'No internet service': 0})
    df['online_backup_bool'] = df.online_backup.map({'Yes': 1, 'No': 0, 'No internet service': 0})
    
    # Get dummies for non-binary categorical variables
    df_dummy = pd.get_dummies(df[['multiple_lines', \
                              'online_security', \
                              'online_backup', \
                              'device_protection', \
                              'tech_support', \
                              'streaming_tv', \
                              'streaming_movies', \
                              'contract_type', \
                              'internet_service_type', \
                              'payment_type']], dummy_na=False, drop_first=True)
    df = pd.concat([df, df_dummy],axis=1)
    
    # encode number_relationships by utilizing information from dependents_encoded and partner_encoded
    df['number_relationships'] = df['dependents_encoded'] + df['partner_encoded']

    # encode number_online_services by utilizing information from online_security_encoded and online_backup_encoded
    df['number_online_services'] = df['online_security_bool'] + df['online_backup_bool']

    # encode tenure in years (rounded down) by utilizing information from tenure (currently stored in months)
    df['yearly_tenure'] = df.tenure.apply(lambda x: math.floor(x/12))

    # encode has_internet
    df['has_internet'] = df.internet_service_type.apply(lambda x: 0 if x == 'None' else 1)
    
    # make another column for additional online services
    df['additional_services'] = (df[['online_security','online_backup', 'device_protection', 'tech_support', 'streaming_tv',
                                 'streaming_movies']] == 'Yes').sum(axis=1)

    return df

## split data

In [65]:
train_validate, test = train_test_split(df, test_size=.2, 
                                        random_state=123, 
                                        stratify=df.churn)

train, validate = train_test_split(train_validate, test_size=.3, 
                                   random_state=123, 
                                   stratify=train_validate.churn)

In [66]:
print(train.shape)
print(validate.shape)
print(test.shape)

(3937, 54)
(1688, 54)
(1407, 54)


In [60]:
def split_telco_data(df):
    '''
    This function performs split on telco data, stratify churn.
    Returns train, validate, and test dfs.
    '''
    train_validate, test = train_test_split(df, test_size=.2, 
                                        random_state=123, 
                                        stratify=df.churn)
    train, validate = train_test_split(train_validate, test_size=.2, 
                                   random_state=123, 
                                   stratify=train_validate.churn)
    return train, validate, test

In [61]:
def prep_telco_data(df):
    df = clean_telco_data(df)
    train, validate, test = split_telco_data(df)
    
    return train, validate, test